In [51]:
from ultralytics import YOLO
import os
import geopandas as gpd
from shapely.geometry import Polygon

In [34]:
model = YOLO("runs/detect/train2/weights/best.pt")  # pretrained YOLOv8n model

In [35]:
image_file = "../data/tilesZugeschnittenUndJsonZurGeoRef/tile_1024_12800.tif"

In [36]:
results = model(image_file)
print(results[0].boxes[0])


image 1/1 C:\Users\phili\Documents\Studium\Master_Geographie\5_Semester\S_Deep_Learning\beet-segmentation\src\..\data\tilesZugeschnittenUndJsonZurGeoRef\tile_1024_12800.tif: 512x512 1 sugar beet, 61.8ms
Speed: 3.0ms preprocess, 61.8ms inference, 3.0ms postprocess per image at shape (1, 3, 512, 512)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.2744])
data: tensor([[3.2210e+02, 2.1581e+02, 3.6341e+02, 2.6650e+02, 2.7444e-01, 0.0000e+00]])
id: None
is_track: False
orig_shape: (512, 512)
shape: torch.Size([1, 6])
xywh: tensor([[342.7574, 241.1524,  41.3069,  50.6885]])
xywhn: tensor([[0.6694, 0.4710, 0.0807, 0.0990]])
xyxy: tensor([[322.1039, 215.8082, 363.4108, 266.4966]])
xyxyn: tensor([[0.6291, 0.4215, 0.7098, 0.5205]])


In [39]:
for result in results:
    boxes = result.boxes
    pixel_xmin = boxes.xyxy[0][0].item()
    pixel_ymin = boxes.xyxy[0][1].item()
    pixel_xmax = boxes.xyxy[0][2].item()
    pixel_ymax = boxes.xyxy[0][3].item()

In [40]:
print(pixel_xmin, pixel_ymin, pixel_xmax, pixel_ymax)

322.10394287109375 215.80816650390625 363.41082763671875 266.49664306640625


In [41]:
from osgeo import gdal
import numpy as np

def pixel_to_geo(pixel_x, pixel_y, geotransform):
    geo_x = geotransform[0] + pixel_x * geotransform[1] + pixel_y * geotransform[2]
    geo_y = geotransform[3] + pixel_x * geotransform[4] + pixel_y * geotransform[5]
    return geo_x, geo_y

In [42]:
# Example usage
dataset = gdal.Open(image_file)
if dataset is None:
    raise FileNotFoundError("Image file not found")

# Get geotransform
geotransform = dataset.GetGeoTransform()

In [43]:
print(geotransform)

(576497.2407970831, 0.00040246724450860487, 0.0, 5516317.9808907695, 0.0, -0.00040246724450860487)


In [45]:
# Convert pixel coordinates to geographic coordinates
geo_xmin, geo_ymin = pixel_to_geo(pixel_xmin, pixel_ymin, geotransform)
geo_xmax, geo_ymax = pixel_to_geo(pixel_xmax, pixel_ymax, geotransform)
print("Geographic coordinates:", geo_xmin, geo_ymin, geo_xmax, geo_ymax)

# Close the dataset
dataset = None

Geographic coordinates: 576497.3704333694 5516317.894035052 576497.3870580376 5516317.8736346


In [49]:
# Create a bounding box as a Shapely Polygon object
bounding_box = Polygon([(geo_xmin, geo_ymin), (geo_xmax, geo_ymin), (geo_xmax, geo_ymax), (geo_xmin, geo_ymax)])

# Print the bounding box
print("Bounding box as a Shapely Polygon object:", bounding_box)

Bounding box as a Shapely Polygon object: POLYGON ((576497.3704333694 5516317.894035052, 576497.3870580376 5516317.894035052, 576497.3870580376 5516317.8736346, 576497.3704333694 5516317.8736346, 576497.3704333694 5516317.894035052))


In [52]:
# Create a GeoDataFrame with the bounding box
gdf = gpd.GeoDataFrame(geometry=[bounding_box])

# Define the output file path
output_file = "bounding_box.geojson"

# Export the GeoDataFrame to a GeoJSON file
gdf.to_file(output_file, driver='GeoJSON')

print("Bounding box exported to:", output_file)

Bounding box exported to: bounding_box.geojson
